In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import TfidfVectorizer
import re
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

from sklearn.metrics import accuracy_score, auc, roc_curve, roc_auc_score

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data_news = pd.read_csv("../input/data-viec-lam-uv/data_viec_lam_uv.csv")
data_news = data_news.drop(data_news.loc[data_news['Tiêu đề tin '].isnull()].index)

print(data_news.columns)



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer


def get_stopwords_list(stop_file_path):
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(frozenset(stop_set))
    
#from nltk.corpus import stopwords

class LemmaTokenizer(object):
    def __init__(self,stopwords):
        # lemmatize text - convert to base form
        self.wnl = WordNetLemmatizer()
        # creating stopwords list, to ignore lemmatizing stopwords
        self.stopwords = stopwords
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if t not in self.stopwords]
    
stopwords = get_stopwords_list("../input/stop-words-in-28-languages/vietnamese.txt")



In [ ]:
def change_titles(x):
    x = x.strip().lower()
    if 'bất động sản' in x:
        return 'bất động sản'
    elif 'bđs' in x:
        return 'bất động sản'
    elif  'tư vấn' in x:
        return'tư vấn'
    elif  'chăm sóc khách hàng' in x:
        return 'chăm sóc khách hàng'
    elif  'kế toán' in x:
        return 'kế toán'
    elif  'kinh doanh' in x:
        return 'kinh doanh'
    elif  'xuất nhập khẩu' in x:
        return 'xuất nhập khẩu'
    elif 'logistics' in x:
        return 'xuất nhập khẩu'
    elif 'khai báo hải quan' in x:
        return 'xuất nhập khẩu'
    elif 'nhập liệu chứng từ' in x:
        return 'xuất nhập khẩu'
    elif  'admin hành chính' in x:
        return 'hành chính'
    elif 'nhân sự' in x:
        return 'nhân sự'
    elif 'làm thêm' in x:
        return 'làm thêm'
    elif 'bán thời gian' in x:
        return 'làm thêm'
        
    elif 'marketing' in x:
        return 'marketing'
    elif 'seo' in x:
        return 'marketing'
    elif 'pr' in x:
        return 'marketing'
    elif 'công nghệ thông tin' in x:
        return 'công nghệ thông tin'
    elif 'tester' in x:
        return 'công nghệ thông tin'
    elif 'công nhân' in x:
        return 'công nhân'
    elif 'admin văn phòng' in x:
        return 'quản trị văn phòng'
   
   
    elif 'it' in x:
        return 'công nghệ thông tin'
    elif 'lập trình viên' in x:
        return 'công nghệ thông tin'
    elif 'php' in x:
        return 'công nghệ thông tin'
    elif 'lập trình' in x:
        return 'công nghệ thông tin'
        
    elif 'phục vụ' in x:
        return 'phục vụ'
    elif 'trợ lý' in x:
        return 'trợ lý'
    elif 'thư ký' in x:
        return 'thư ký'
        
    elif 'dược sĩ nghiên cứu' in x:
        return 'y tế'
    elif 'trình dược viên' in x:
        return 'y tế'
    elif 'dược phẩm' in x:
        return 'y tế'
    elif 'lễ tân' in x:
        return 'lễ tân'
    elif 'thủ kho dược' in x:
        return 'y tế'
    elif 'bác sĩ part time' in x:
        return 'y tế'
    elif 'y tế công ty' in x:
        return 'y tế'
        
    elif 'bán hàng' in x:
        return 'bán hàng'
    elif 'kiểm toán' in x:
        return 'kế toán'
    elif 'bảo hiểm' in x:
        return 'bảo hiểm'
        
    elif 'kỹ thuật điện' in x:
        return 'cơ khí điện tử'
    elif 'vận hành máy' in x:
        return 'cơ khí điện tử'
    elif 'kỹ sư cơ khí' in x:
        return 'cơ khí điện tử'
    elif 'cơ khí' in x:
        return 'cơ khí điện tử'
    elif 'kỹ sư mep' in x:
        return 'cơ khí điện tử'
    elif 'kỹ sư điện'in x:
        return 'cơ khí điện tử'
    elif 'lắp ráp cài đặt máy tính' in x:
        return 'cơ khí điện tử'
        
        
    elif 'thợ phụ điện công trình' in x:
        return 'xây dựng'
    elif 'kỹ sư xây dựng' in x:
        return 'xây dựng'
    elif 'kỹ thuật xây dựng' in x:
        return 'xây dựng'
    elif 'kỹ sư thiết kế xây dựng ' in x:
        return 'xây dựng'
    elif 'kiến trúc sư' in x:
        return 'xây dựng'
    elif 'thợ xây' in x:
        return 'xây dựng'
    elif 'giám sát công trình' in x:
        return 'xây dựng'
    elif 'giám sát xây dựng' in x:
        return 'xây dựng'
    elif 'dự toán xây dựng' in x:
        return 'xây dựng'
    elif 'kỹ sư cầu đường' in x:
        return 'xây dựng'
    elif 'kỹ sư kinh tế xây dựng' in x:
        return 'xây dựng'
    elif 'kỹ sư an toàn xây dựng' in x:
        return 'xây dựng'
        
    elif 'giáo viên' in x:
        return 'giáo dục'
    elif 'dạy thêm' in x:
        return 'giáo dục'
    elif 'giảng viên' in x:
        return 'giáo dục'
    elif 'gia sư' in x:
        return 'giáo dục'
    elif 'trợ giảng' in x:
        return 'giáo dục'
    elif 'tài xế' in x:
        return 'lái xe'
    elif 'lái xe' in x:
        return 'lái xe'
    elif 'telesale' in x:
        return 'sale'
    elif 'sale' in x:
        return 'sale'
        
    elif 'chuyên viên thương mại điện tử' in x:
        return 'thương mại điện tử'
        
    elif 'cán bộ kỹ thuật hiện trường' in x:
        return 'xây dựng'
    elif 'thiết kế' in x:
        return 'thiết kế'
    elif 'họa viên kiến trúc' in x:
        return 'thiết kế'
    elif 'vẽ autocad 2d' in x:
        return 'thiết kế'
    elif '2d artist' in x:
        return 'thiết kế'
    elif 'designer' in x:
        return 'thiết kế'
    elif 'họa viên 3d nội thất' in x:
        return 'thiết kế'
        
    elif 'điều phối đơn hàng' in x:
        return 'quản lý vận hành'
    elif 'tiếp nhận đơn hàng' in x:
        return 'quản lý vận hành'
    elif 'xử lý đơn hàng' in x:
        return 'quản lý vận hành'
    elif 'quản lý đơn hàng may mặc' in x:
        return 'quản lý vận hành'
    elif 'content' in x:
        return 'biên tập nội dung'

    elif 'hỗ trợ khách hàng' in x:
        return 'dịch vụ khách hàng'
    elif 'customer service' in x:
        return 'dịch vụ khách hàng'
    
    elif 'kỹ sư điện tử viễn thông' in x:
        return 'điện tử viễn thông'
    elif 'pháp chế'in x:
        return 'pháp chế'
    elif 'quản lý chuỗi cửa hàng bán lẻ' in x:
        return 'quản lý giám sát'
    elif 'đóng gói hàng hoá' in x:
        return 'vận hành sản xuất'
    elif 'spa' in x:
        return 'spa'
    elif 'part time' in x:
        return 'làm thêm'
    elif 'bảo vệ' in x:
        return 'bảo vệ'
    elif 'thủy sản' in x:
        return 'thuỷ sản'
    elif 'triển khai phần mềm' in x:
        return 'công nghệ thông tin'
    
    elif 'quản lý hành chính' in x:
        return 'hành chính'
    elif 'tài chính' in x:
        return 'tài chính'
    elif 'thực phẩm' in x:
        return 'thực phẩm'
    elif 'giao hàng' in x:
        return 'giao hàng'
    elif 'phiên dịch' in x:
        return 'phiên dịch'
    elif 'thu mua vật tư' in x:
        return 'mua hàng'
    elif 'quản trị nhân lực' in x:
        return 'nhân sự'
    elif 'sản xuất' in x:
        return 'sản xuất'
    return x.strip()

In [ ]:
def processing_title(x):
    x = x.strip().lower()
    x = re.sub("([\(\[]).*?([\)\]])", "", x)
    return x
    

In [ ]:
data_news['Lĩnh vực '] = data_news['Lĩnh vực '].apply(lambda s:s.strip().replace('nhân viên','')
                                    .replace('chuyên viên','').replace('không cần kinh nghiệm','')).apply(change_titles).apply(lambda s:s.strip()+' ')

In [ ]:
classes = data_news['Lĩnh vực '].value_counts()[:10]
keys = classes.keys().to_list()

data_news = data_news[data_news['Lĩnh vực '].isin(keys)]

In [ ]:
carrier = data_news['Ngành nghề '].apply(lambda s:s.strip()+' ').apply(lambda s:s.replace(', ',' ').replace('- ',''))

title = data_news['Tiêu đề tin '].apply(lambda s:str(s).lower().strip().replace('- ','').replace(':','').replace('nhân viên','')
                                .replace('việc làm','').replace('không yêu cầu bằng cấp','')).apply(lambda s:processing_title(s))    
total = carrier + title
vector = TfidfVectorizer(#max_df=0.5,         # drop words that occur in more than X percent of documents
                             #min_df=1,      # only use words that appear at least X times
                             stop_words=None, # remove stop words
                             lowercase=True, # Convert everything to lower case 
                             use_idf=True,   # Use idf
                             norm=u'l2',     # Normalization
                             smooth_idf=True # Prevents divide-by-zero errors
                            )
vector.fit(total)

In [ ]:
X = total
y = data_news['Lĩnh vực ']


In [ ]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
le = LabelEncoder()
#y_enc=mlb.fit_transform(y)
Y_enc = (le.fit_transform(y.values))
Y_enc=Y_enc.reshape(-1,)
print(len(Y_enc))
X_train_words, X_test_words, y_train, y_test = train_test_split(X, Y_enc, test_size=0.1, random_state=10)
X_train = vector.transform(X_train_words)
X_train = X_train.toarray()

X_test = vector.transform(X_test_words)
X_test = X_test.toarray()


In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression(class_weight = 'balanced',l1_ratio=0,n_jobs = -1,verbose=1, C=0.9)

train_preds = logistic.fit(X_train, y_train).predict(X_train)
test_preds = logistic.predict(X_test)

print('Train acc: {0}'.format(accuracy_score(y_train, train_preds)))
print('Test acc: {0}'.format(accuracy_score(y_test, test_preds)))

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# rfc = RandomForestClassifier(n_jobs = -1,oob_score = True,random_state = 10)
# rfc.fit(X_train, y_train)
# train_p = rfc.predict(X_train)
# test_p = rfc.predict(X_test)
# print('Train acc: {0}'.format(accuracy_score(y_train, train_p)))
# print('Test acc: {0}'.format(accuracy_score(y_test, test_p)))


In [ ]:
#classifiaction.
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

#evaluation metrics
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error # for regression
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score  # for classification
models=[LinearSVC(),SVC(kernel='rbf'),KNeighborsClassifier(),RandomForestClassifier(n_estimators = 20,max_depth = 5,n_jobs = -1,oob_score = True,random_state = 0),
        DecisionTreeClassifier()]
model_names=['LinearSVM','rbfSVM','KNearestNeighbors','RandomForestClassifier','DecisionTree'
            ]

acc=[]
d={}
clf =[]
for model in range(len(models)):
    clf=models[model]
    clf.fit(X_train, y_train)
    pred=clf.predict(X_test)
    acc.append(accuracy_score(pred,y_test))
     
d={'Modelling Algo':model_names,'Accuracy':acc}

In [ ]:
d

In [ ]:
print(X_train_pca.shape)

In [ ]:
# from tensorflow.keras import layers,Input,Model

# inputs = Input(shape = (1647, 15407))
# x = layers.Dense(64,activation = 'relu')(inputs)
# x = layers.Dense(128,activation = 'relu')(x)
# outputs = layers.Dense(50,activation = 'softmax')(x)

# model = Model(inputs = inputs,outputs = outputs)
# model.summary()
# model.compile(loss = 'SparseCategoricalCrossentropy',optimizer = 'adam',metrics = [f1_score])

# model.fit(X_train,y_train,
#          batch_size = 32,
#          epochs = 10,
#          verbose = 1,
#          validation_split = 0.2)



In [ ]:
preds_data = {'Current Position Requirments': [], 'Current Carrier': [], 'Alternative 1': [], 'Alternative 2': []}
y_preds_proba = logistic.predict_proba(X_test)
print(len(y_preds_proba))
print(len(y_test))
counter = 0 
for idx, (pred_row, true_job_position) in enumerate(zip(y_preds_proba, y_test)):
    class_preds = np.argsort(pred_row)
    # delete true class
    for i in [-1, -2]:
        if class_preds[i] == true_job_position:
            class_preds=np.delete(class_preds,i)
    # getting other 2 highest job predictions         
    top_classes = class_preds[-2:]
    # obtaining class name string from int label 
    class_names = le.inverse_transform(top_classes)
    true_job_position_name = le.inverse_transform([true_job_position])
    # saving to dict
    preds_data['Current Position Requirments'].append(X_test_words.iloc[idx])
    preds_data['Current Carrier'].append(true_job_position_name[0])
    preds_data['Alternative 1'].append(class_names[1])
    preds_data['Alternative 2'].append(class_names[0])

    
    counter +=1

In [ ]:
preds_df = pd.DataFrame.from_dict(preds_data)
print(len(preds_df))
preds_df[:10]

In [ ]:
'Nhan vien' in 'Nhan vien kinh doanh'